# Visualización Avanzada
En el presente trabajo se analizarán los datos de COVID-19 que se obtienen a partir de la declaración de los casos a la Red Nacional de Vigilancia Epidemiológica (RENAVE) a través de la plataforma informática vía Web SiViES (Sistema de Vigilancia de España) que gestiona el Centro Nacional de Epidemiología (CNE). Estos datos corresponden al periodo comprendido entre el inicio de la pandemia (enero-2020) hasta el 28 de marzo de 2022.

Para realizar algunos cálculos más específicos utilizaremos datos del INE, específicamente de población por provincia y franja etaria de los últimos tres años (2020, 2021 y 2022) y los datos de polígonos que correspoden a las representaciones gráficas de las provincias de España.

**Para empezar, importamos las librerías que utilizaremos para este análisis.**

In [ ]:
%pip install geopandas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from datetime import date
from datetime import datetime
import plotly.express as px
import plotnine
from plotnine import *
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import folium

### Lectura de datasets
Realizamos la lectura de los datasets de Covid y provincias (previamente descargados) y realizamos unos primeros ajustes a los mismos.

Los archivos se encuentran en el mismo directorio del presente notebook.

In [ ]:
covid = pd.read_csv('https://cnecovid.isciii.es/covid19/resources/casos_hosp_uci_def_sexo_edad_provres.csv')

In [ ]:
#covid = pd.read_csv(r'casos_hosp_uci_def_sexo_edad_provres.csv')
prv = pd.read_csv(r'provinces_es.csv')
geo_prv = r'SP_provincias.geojson'
map = gpd.read_file(geo_prv)

In [ ]:
map['postal_code'] = pd.to_numeric(map['CC_2'])
map.drop(['CC_2', 'NAT2018'], axis = 'columns', inplace=True)
map.rename(columns={'NAME_1':'comunidad_autonoma',
                    'NAME_2':'provincia'}, inplace = True)

**Realizamos una pre-visualización del contenido**

In [ ]:
covid.head()

In [ ]:
prv.head()

In [ ]:
map.head()

**Empezamos uniendo nuestros datasets de covid y provincias**

In [ ]:
cov_prv = covid.merge(prv, 
                      how='left', 
                      left_on='provincia_iso', 
                      right_on='code', 
                      suffixes=('_cov', '_prv'))

**Realizamos algunos formateos a los datos, para que nos ayuden en las visualizaciones posteriores**

In [ ]:
cov_prv['periodo'] = cov_prv['fecha'].str.slice(0, 4)
cov_prv['periodo'] = pd.to_numeric(cov_prv['periodo'])
cov_prv['fecha'] = pd.to_datetime(covid['fecha'], 
                                  format = '%Y-%m-%d', 
                                  errors = 'coerce')
cov_prv['semana'] = cov_prv['fecha'].dt.week

cov_prv.drop(['name', 'phone_code', 'iso2'], 
             axis = 'columns', inplace=True)
cov_prv.rename(columns={'grupo_edad': 'franja_etaria'}, 
               inplace=True)
cov_prv.head()

**Realizamos la importación de los datos de población de cada provincia, correspondientes a los años 2020, 2021 y 2022.**

**Luego, unimos las tres fuentes en un único dataset.**

In [ ]:
pob_esp2020 = pd.read_csv(r'PoblacionEspana-Provincia_2020.csv', 
                      sep=';', 
                      encoding='latin-1')
pob_esp2021 = pd.read_csv(r'PoblacionEspana-Provincia_2021.csv', 
                      sep=';', 
                      encoding='latin-1')
pob_esp2022 = pd.read_csv(r'PoblacionEspana-Provincia_2022.csv', 
                      sep=';', 
                      encoding='latin-1')
pob_esp = pd.concat([pob_esp2020, pob_esp2021, pob_esp2022])
pob_esp.head(3)

**A continuación, realizamos algunos formateos al dataset para dejarlo a punto para el análisis**

In [ ]:
pob_esp.rename(columns={'Edad Simple': 'edad', 
                        'Provincias': 'provincia', 
                        'Sexo':'sexo', 
                        'Periodo':'periodo', 
                        'Total':'total'}, 
               inplace=True)

In [ ]:
pob_esp['total'] = pob_esp['total'].str.replace(',','|')
pob_esp['total'] = pob_esp['total'].str.replace('.','')
pob_esp['total'] = pob_esp['total'].str.replace('|','.')
pob_esp['total'] = pob_esp['total'].astype(float)

In [ ]:
pob_esp['postal_code'] = pob_esp['provincia'].str.slice(0, 2)
pob_esp['postal_code'] = pd.to_numeric(pob_esp['postal_code'])

In [ ]:
pob_esp.loc[pob_esp['sexo'] == 'Hombres', 'sexo'] = 'H'
pob_esp.loc[pob_esp['sexo'] == 'Mujeres', 'sexo'] = 'M'

In [ ]:
datos = pob_esp['edad'].str.split(expand=True)
datos.columns = ['edad2', 'edad3', 'edad4', 'edad5']
pob_esp = pd.concat([pob_esp, datos], axis=1)
pob_esp.drop(['edad3', 'edad4', 'edad5'], axis = 'columns', inplace=True)
pob_esp['edad2'] = pd.to_numeric(pob_esp['edad2'])
pob_esp.head(3)

In [ ]:
def franja_etaria(edad):
    if edad < 10:
        franja = '0-9'
    elif edad < 20:
        franja = '10-19'
    elif edad < 30:
        franja = '20-29'
    elif edad < 40:
        franja = '30-39'
    elif edad < 50:
        franja = '40-49'
    elif edad < 60:
        franja = '50-59'
    elif edad < 70:
        franja = '60-69'
    elif edad < 80:
        franja = '70-79'
    else:
        franja = '80+'
    return franja

In [ ]:
pob_esp['franja_etaria'] = pob_esp['edad2'].apply(franja_etaria)

In [ ]:
separado = pob_esp['periodo'].str.split(' ', expand=True)
pob_esp['periodo']= separado[4]
pob_esp['periodo'] = pd.to_numeric(pob_esp['periodo'])
pob_esp.head(3)

**A continuación, agrupamos los datos de poblacion por año y provincia, para cruzarlo con nuestro dataset de casos de Covid.**

In [ ]:
poblacion = pob_esp.groupby(['periodo', 
                             'provincia', 
                             'postal_code', 
                            ]).agg({'total':'sum'}).reset_index()
poblacion.head()

### Incidencia Acumulada a 14 días por cada 100.000 habitantes
Calculamos la IA a 14 días por cada cien mil habitantes.

In [ ]:
sum_cov = cov_prv.groupby(['fecha', 
                           'periodo']).agg({'num_casos':'sum'})

In [ ]:
def buildLaggedFeatures(s, lag=2, dropna=True):
    if type(s) is pd.DataFrame:
        new_dict={}
        for col_name in s:
            new_dict[col_name]=s[col_name]
            for l in range(1,lag+1):
                new_dict['%s_%d' %(col_name,l)]=s[col_name].shift(l)
        res=pd.DataFrame(new_dict,index=s.index)

    elif type(s) is pd.Series:
        the_range=range(lag+1)
        res=pd.concat([s.shift(i) for i in the_range],axis=1)
        res.columns=['_%d' %i for i in the_range]
    else:
        print ('Only works for DataFrame or Series')
        return None
    if dropna:
        return res.dropna()
    else:
        return res

In [ ]:
casos_trend = buildLaggedFeatures(sum_cov,lag=13,dropna=False)

In [ ]:
casos_trend = casos_trend.reset_index()

In [ ]:
casos_trend2 = pd.melt(casos_trend, 
                       id_vars=['fecha', 'periodo'], 
                       value_vars=['num_casos', 'num_casos_1', 'num_casos_2','num_casos_3', 'num_casos_4', 
                                   'num_casos_5', 'num_casos_6', 'num_casos_7', 'num_casos_8', 'num_casos_9', 
                                   'num_casos_10', 'num_casos_11', 'num_casos_12', 'num_casos_13'])

In [ ]:
casos_trend2 = casos_trend2.groupby(['fecha', 'periodo']).agg({'value':'sum'}).reset_index()
pob = poblacion.groupby(['periodo']).agg({'total':'sum'}).reset_index()
casos_trend_pob = casos_trend2.merge(pob, how='left', left_on='periodo', right_on='periodo')
casos_trend_pob['ia14d'] = casos_trend_pob['value']/casos_trend_pob['total']*100000
casos_trend_pob.head()

### Incidencia acumulada total por cada cien habitantes
Analizaremos de forma gráfica la incidencia por cada cien habitantes durante toda la pandemia.

Para ello, pintaremos en un mapa las distintas provincias de España indicando este indicador en una escala de colores.

In [ ]:
casos_pc = cov_prv.groupby(['periodo', 
                          'postal_code']).agg({'num_casos':'sum', 
                                                'num_hosp':'sum', 
                                                'num_uci':'sum', 
                                                'num_def':'sum'
                                               }).reset_index()


In [ ]:
casos_pob = pd.merge(casos_pc, 
              poblacion, 
              how='left', 
              left_on=['periodo', 'postal_code'], 
              right_on = ['periodo', 'postal_code'], 
              suffixes=('_x', '_y')
             )

In [ ]:
casos_pob = casos_pob.groupby(['postal_code']).agg({'num_casos':'sum', 
                                                    'num_hosp':'sum', 
                                                    'num_uci':'sum', 
                                                    'num_def':'sum',
                                                    'total':'mean'
                                                   }).reset_index()

In [ ]:
casos_pob['incidencia'] = casos_pob['num_casos']/casos_pob['total']*100
casos_pob['incidencia'] = casos_pob['incidencia'].round(2)
casos_pob['total'] = casos_pob['total'].round(2)

In [ ]:
casos_geo = casos_pob.merge(map, 
                            how='left', 
                            left_on='postal_code', 
                            right_on='postal_code')
casos_geo.dropna(subset=['geometry'], inplace=True)
casos_geo.rename(columns={'total':'poblacion'}, inplace = True)
gdf = gpd.GeoDataFrame(casos_geo, geometry=casos_geo.geometry)

In [ ]:
!pip install folium matplotlib mapclassify

In [ ]:
# IA Total por cada cien habitantes
m = gdf.explore(
     column="incidencia",  
     scheme="naturalbreaks",  
     legend=True, 
     k=10, 
     legend_kwds=dict(colorbar=True), 
     name="countries", 
     tooltip=['comunidad_autonoma', 'provincia', 'incidencia'],
     cmap='Oranges'
)

folium.TileLayer('stamentoner').add_to(m)  
#folium.LayerControl().add_to(m) 

m  # show map

Podemos observar en el mapa, cuáles son las provincias con mayor incidencia acumulada.

Por ejemplo, provincias como Navarra, Guipúzcoa, Lleida o Palencia, cuyas IA son visiblemente mayores que las demás regiones geográficas.


### Conclusiones
- La pandemia ha presentado incrementos importantes de casos en periodos específicos a lo largo del tiempo, identificándose como olas de la pandemia. La sexta (comprendida entre octubre de 2021 y marzo de 2022) especialmente ha tenido un pico importante, superando ampliamente el umbral de riesgo muy alto.
- Cabe destacar una marcada diferencia en casos entre hombres y mujeres durante la última ola, cuyos números femeninos superan a los casos masculinos.
- La mayor cantidad de casos se concentró en la franja de jóvenes y adultos (0 a 49 años), esto se verifica también analizando el ratio de casos de población por franja etaria. 
- Teniendo en cuenta ratios de hospitalizaciones, podemos decir que la franja más afectada es la de 60 años o más. Cabe mencionar también que en casi todas las franjas etarias, vemos un mayor ratio de hombres hospitalizados, incrementándose a mayor edad. 
- En ingreso a UCI notamos mayores ratios en hombres que en mujeres, especialmente en el rango 60-79 años.
- La letalidad es claramente mayor en la franja de 60 años en adelante, siendo mayor en hombres que en mujeres.
- Analizando la incidencia acumulada por cada cien habitantes, destacamos en el top 3 con mayor IA a las provincias de Navarra, Guipúzcoa y Lleida.